<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_NER_Training_(blank_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy==3.0.6
# !python -m spacy download zh_core_web_sm
# !python -m spacy download zh_core_web_trf

     |████████████████████████████████| 12.8 MB 70 kB/s 
     |████████████████████████████████| 9.1 MB 37.5 MB/s 
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.1.1
    Uninstalling spacy-3.1.1:
      Successfully uninstalled spacy-3.1.1


In [15]:
TRAIN_DATA = [
  (
    '現時恒指已跌穿重要支持位26500點來看，港股繼續尋底的機會是頗高的。',
    {
        'entities':
        [
         (2,4,'FIN'),(9,12,'FIN'),(21,23,'FIN'),(25,27,'FIN')
        ]
    }
  ),
  (
    '恆生指數和創業板指數做好。',
    {
      'entities': 
      [
        (0,4,'FIN'),(5,10,'FIN'),(10,12,'FIN')          
      ]
    }
  ),  
  (
    '中電、匯控、和黃8月暴升8.5%。',
    {
      'entities': 
      [
        (0,2,'STOCK'),(3,5,'STOCK'),(6,8,'STOCK'),(10,12,'FIN'),(12,16,'PERCENT')        
      ]
    }
  ),
  (
    '恆指小雙底9月盼反彈。中電、匯控、和黃8月齊齊跌8.5%金融海嘯後最差。',
    {
      'entities': 
      [
        (0,2,'FIN'),(2,5,'FIN'),(8,10,'FIN'),(11,13,'STOCK'),(14,16,'STOCK'),
        (17,19,'STOCK'),(19,21,'DATE'),(24,28,'PERCENT'),(28,32,'EVENT') 
      ]
    }
  ),     
  (
    '匯控(00005)將於下周一（2日）公布2021年度中期業績。',
    {
        'entities':
        [
          (0,2,'STOCK'),(20,24,'DATE'),(26,30,'FIN')
        ]
    }
  ),
  (
    '查看香港最新的股市報價、記錄、新聞及其他重要資訊，助你賣買股票及投資。',
   {
      'entities':
      [
        (2,4,'GPE'),(7,9,'FIN'),(9,11,'FIN')
      ]
   }
  ),
  (
    '匯控低開4%。港股收市跌。',
    {
        'entities': [(0,2,'STOCK'),(4,6,'PERCENT'),(7,9,'FIN')]
    }
  ),
  (
    '港股高開173點。',
   {
       'entities': [(0,2,'FIN'),(2,4,'FIN')]
   }
  ),
  (
    '報道引述和黃內部電郵稱',{'entities':[(4,6,'STOCK')]}
  ),
  (
    '匯控將在第四季開始進行股份回購',{'entities':[(0,2,'STOCK')]}
  ),
    (
    '花旗發表研究報告，下調匯控（0005）每股盈利預測',{'entities':[(11,13,'STOCK')]}
  ),
    (
    '和黃將恢復派息',{'entities':[(0,2,'STOCK')]}
  )
]

import spacy
import random
from spacy.training.example import Example

# Create a blank "zh" model
nlp = spacy.blank("zh")

# Create a new entity recognizer and add it to the pipeline
ner = nlp.create_pipe("ner")
nlp.add_pipe("ner")

# Add the labels to the entity recognizer
for texts, annotations in TRAIN_DATA:
    for entity in annotations['entities']:
      ner.add_label(entity[2])

print(f'ner: {ner.labels}')

# Start the training
nlp.begin_training()

# Loop for n iterations
for itn in range(100):
    # Shuffle the training data
    random.shuffle(TRAIN_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAIN_DATA, size=2):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]

        # Update the model
        # nlp.update(texts, annotations, losses=losses)
        example = []
        for i in range(len(texts)):
          doc = nlp.make_doc(texts[i])
          example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(example,drop=0.2,losses=losses)

    print(losses)

ner: ('DATE', 'EVENT', 'FIN', 'GPE', 'PERCENT', 'STOCK')
{'ner': 221.19360780715942}
{'ner': 146.77301014959812}
{'ner': 69.0825639208997}
{'ner': 58.543829676484165}
{'ner': 90.27809873409569}
{'ner': 51.916197788581485}
{'ner': 44.15940956629129}
{'ner': 39.93475238528754}
{'ner': 66.9112047561066}
{'ner': 51.85204359114712}
{'ner': 72.00019167200514}
{'ner': 42.60388408492054}
{'ner': 33.765277643070476}
{'ner': 30.558502333092036}
{'ner': 22.996520634998298}
{'ner': 18.461227832619638}
{'ner': 48.664820852752115}
{'ner': 10.190555913181857}
{'ner': 17.680808113626913}
{'ner': 19.39134885446869}
{'ner': 10.768295012267625}
{'ner': 15.52949576041433}
{'ner': 8.197179721317076}
{'ner': 5.873448118207171}
{'ner': 38.98599823271263}
{'ner': 13.782779301081657}
{'ner': 9.425609663986018}
{'ner': 9.009621802365753}
{'ner': 12.86690532457975}
{'ner': 9.667514180513765}
{'ner': 4.448646597328378}
{'ner': 5.6281142242873194}
{'ner': 1.815415657384919}
{'ner': 7.476904520965401}
{'ner': 10.01

In [16]:
from spacy import displacy

texts = [
  '恆指上升3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。',
  '中電在假期前做好帶動，恒指單日暴升961.22點。',
  '受美股收跌拖累，匯控、和記跌超過2%。',
  '匯控下跌2%。',
  '日本、新加坡、澳洲股市單日暴升961.22點。'
]

for text in texts:
  doc = nlp(text)
  displacy.render(doc,jupyter=True,style="ent")
